In [1]:
import utility_class
from utility_class import Mycv
from utility_class import Graph
from utility_class import Coordinate
import numpy as np
import cv2
import os
from make_robot_graph import *
import os
from scipy.spatial import distance

In [2]:
# 保存したファイルを読み込む
joint_heatmap_tf = np.load('dataset/joint_heatmap_tf.npy')
endeffector_heatmap_tf = np.load('dataset/endeffector_heatmap_tf.npy')
i=26031
image_path=f'dataset/joint_dataset/outline/outline_{str(i).zfill(8)}.png'
image_cv=cv2.imread(image_path)

robot_array=Mycv.binarize_to_fill_image(image_cv,0,None)#白塗りつぶし画像を作成
gray = cv2.cvtColor(robot_array, cv2.COLOR_BGR2GRAY)
# 2値化（白:255 -> 1, 黒:0 -> 0）
binary_array = (gray > 0).astype(int)  # オブジェクトを1、背景を0とする
#cv2.imwrite(f'result/test/black_{str(i).zfill(8)}.png', robot_array)

#3つのグラフ
JG,object_points,joint_set,object_set,joint_points=Graph.new_create_graph_from_points(joint_heatmap_tf,binary_array,flag='joint')
EG,end_points=Graph.new_create_graph_from_points(endeffector_heatmap_tf,binary_array,flag='endeffector')
GG,multi_node=Graph.docking_graph(JG,EG,object_set,joint_set)

#先生のグラフにしてから全域木にする
NG=Graph.teacher_graph(GG,multi_node)
DG = Graph.create_rooted_directed_tree(NG, root_param='Root')

write_dot(DG, "dataset/joint_graph.dot")
print("いけた")

joint_points: [[237 119]
 [270 119]
 [270 129]
 [303 203]
 [350 329]]
ロボット領域にないjointを削除します: []
set,point 45 5
joint同士が近すぎるから繋がねーわ [270 119] [270 129] 10.0
endffector_points: [[171  90]
 [187 137]]
ただ結合しただけ Graph with 7 nodes and 4 edges
今回探すエンドエフェクタ (171, 90)
0.6ルールクリアでジョイントとエンドエフェクタを結んだよ (237, 119) (171, 90) 72.09022125087424 0.9324324324324325
今回探すエンドエフェクタ (187, 137)
0.6ルールクリアでジョイントとエンドエフェクタを結んだよ (237, 119) (187, 137) 53.14132102234569 0.8545454545454545
このノードは複数のエンドエフェクタにつながってたから中間にノード増やすよ JG_0
お前はjointだからスキップ JG_1
エンドエフェクタがない末端のジョイント削除後
 [('EG_(171, 90)', 'JG_(204, 104)'), ('EG_(187, 137)', 'JG_(212, 128)'), ('JG_0', 'JG_1'), ('JG_0', 'JG_(220, 111)'), ('JG_0', 'JG_(224, 123)'), ('JG_1', 'JG_2'), ('JG_2', 'JG_3'), ('JG_3', 'JG_4'), ('JG_(204, 104)', 'JG_(220, 111)'), ('JG_(212, 128)', 'JG_(224, 123)')]
色々あったあと Graph with 11 nodes and 10 edges
複数のエンドエフェクタの元ジョイントは L2
各Gノードから最も遠いノード JG_4 を G_Root と接続しました。
いけた


In [3]:
for node, attrs in DG.nodes(data=True):
    print(f"Node {node}:")
    for attr, value in attrs.items():
        print(f"  {attr}: {value}")
    print("\n\n")

Node Root:
  shape: diamond
  params: {'node': 'Root', 'type': 'link', 'joint': {'type': 'root'}}



Node G_Root:
  shape: box
  params: {'node': 'G_Root', 'type': 'geom', 'geometry': {'primitive': 'box', 'args': {'x': 0.2, 'y': 0.2, 'z': 0.2, 'color': [1, 1, 1]}}, 'original_node': 'JG_(224, 123)'}



Node L6:
  params: {'node': 'L6', 'type': 'link', 'original_node': 'JG_4', 'translation': [0, 0, 1.0], 'relative': True, 'joint': {'type': 'ball'}}
  shape: cylinder



Node L5:
  params: {'node': 'L5', 'type': 'link', 'original_node': 'JG_3', 'translation': [0, 0, 1.0], 'relative': True, 'joint': {'type': 'ball'}}
  shape: cylinder



Node L4:
  params: {'node': 'L4', 'type': 'link', 'original_node': 'JG_2', 'translation': [0, 0, 1.0], 'relative': True, 'joint': {'type': 'ball'}}
  shape: cylinder



Node L3:
  params: {'node': 'L3', 'type': 'link', 'original_node': 'JG_1', 'translation': [0, 0, 1.0], 'relative': True, 'joint': {'type': 'ball'}}
  shape: cylinder



Node L2:
  params: {'